In [1]:
import time

notebook_start_time = time.time()

# Set up environment

In [2]:
import sys
from pathlib import Path


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/decodingml/hands-on-recommender-system.git
    %cd hands-on-recommender-system/


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()

    root_dir = str(Path().absolute())
    print("⛳️ Google Colab environment")
else:
    root_dir = str(Path().absolute().parent)
    print("⛳️ Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

fatal: destination path 'hands-on-recommender-system' already exists and is not an empty directory.
/content/hands-on-recommender-system
Using Python 3.11.11 environment at: /usr
Resolved 198 packages in 675ms
Audited 198 packages in 0.27ms
⛳️ Google Colab environment
Adding the following directory to the PYTHONPATH: /content/hands-on-recommender-system


# Online inference pipeline: Deploying and testing the real-time ML services

In this notebook, we will dig into the inference pipeline and deploy it to Hopsworks as a real-time service.

## 📝 Imports

In [3]:
import warnings

warnings.filterwarnings("ignore")

from loguru import logger

from recsys import hopsworks_integration

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [4]:
project, fs = hopsworks_integration.get_feature_store()

2025-02-19 04:16:52.125 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:13 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213603


# Deploying the ranking inference pipeline


You start by deploying your ranking model. Since it is a CatBoost model you need to implement a `Predict` class that tells Hopsworks how to load the model and how to use it:

In [5]:
ranking_deployment = hopsworks_integration.ranking_serving.HopsworksRankingModel.deploy(
    project=project
)

Uploading: 0.000%|          | 0/4491 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1113 elapsed<00:00 remaining<?

Deployment with the same name already exists. Getting existing deployment...
To create a new deployment choose a different name.
Before making predictions, start the deployment by using `.start()`


Now, we have to explicitly start the deployment:

In [6]:
ranking_deployment.start()

  0%|          | 0/6 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


In [ ]:
# Check logs in case of failure
# ranking_deployment.get_logs(component="transformer", tail=200)

## <span style="color:#ff5f27"> Test the ranking inference pipeline</span>


In [7]:
def get_top_recommendations(ranked_candidates, k=3):
    return [candidate[-1] for candidate in ranked_candidates["ranking"][:k]]

Let's define a dummy test example to test our ranking deployment (only the `customer_id` has to match):

In [8]:
test_ranking_input = [
        {
            "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
            "month_sin": 1.2246467991473532e-16,
            "query_emb": [
                0.214135289,
                0.571055949,
                0.330709577,
                -0.225899458,
                -0.308674961,
                -0.0115124583,
                0.0730511621,
                -0.495835781,
                0.625569344,
                -0.0438038409,
                0.263472944,
                -0.58485353,
                -0.307070434,
                0.0414443575,
                -0.321789205,
                0.966559,
            ],
            "month_cos": -1.0,
        }
    ]

# Test ranking deployment
ranked_candidates = ranking_deployment.predict(inputs=test_ranking_input)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
recommendations

['767687001', '347200001', '632193001']

Check logs in case of failure:

In [ ]:
# ranking_deployment.get_logs(component="transformer", tail=200)

# Deploying the query inference pipeline

In [9]:
query_model_deployment = (
    hopsworks_integration.two_tower_serving.HopsworksQueryModel.deploy(ranking_model_type="ranking")
)

Connection closed.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213603
Connection closed.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213603
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Uploading: 0.000%|          | 0/2971 elapsed<00:00 remaining<?

Deployment with the same name already exists. Getting existing deployment...
To create a new deployment choose a different name.
Before making predictions, start the deployment by using `.start()`


At this point, you have registered your deployment. To start it up you need to run:

In [10]:
query_model_deployment.start()

  0%|          | 0/6 [00:00<?, ?it/s]

Start making predictions by using `.predict()`


In [16]:
query_model_deployment.get_state()

PredictorState(status: 'Running')

In [ ]:
# Check logs in case of failure
# query_model_deployment.get_logs(component="transformer", tail=20)

## <span style="color:#ff5f27"> Testing the inference pipeline </span>

Define a test input example:

In [17]:
data = [
    {
        "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
        "transaction_date": "2022-11-15T12:16:25.330916",
    }
]

Test out the deployment:

In [18]:
ranked_candidates = query_model_deployment.predict(inputs=data)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
recommendations

['636587002', '569996004', '616225015']

Check logs in case of failure:

In [ ]:
# query_model_deployment.get_logs(component="transformer", tail=200)

# <span style="color:#ff5f27"> Stopping the Hopsworks deployments </span>

Stop the deployment when you're not using it.

In [ ]:
ranking_deployment.stop()
query_model_deployment.stop()

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

## <span style="color:#ff5f27"> Inspecting the deployments in Hopsworks UI </span>

View results in [Hopsworks Serverless](https://rebrand.ly/serverless-github): **Data Science → Deployments**

---

In [ ]:
notebook_end_time = time.time()
notebook_execution_time = notebook_end_time - notebook_start_time

logger.info(
    f"⌛️ Notebook Execution time: {notebook_execution_time:.2f} seconds ~ {notebook_execution_time / 60:.2f} minutes"
)

2024-12-24 13:14:20.862 | INFO     | __main__:<module>:4 - ⌛️ Notebook Execution time: 133.44 seconds ~ 2.22 minutes


# <span style="color:#ff5f27">→ Next Steps </span>

The last step is to schedule the materialization jobs.